# 1. Import Modules and Data

In [1]:
# Auto-reload custom modules
%load_ext autoreload
%autoreload 2
import sys
import os

# adds modules to path so that it can be imported using
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import configparser
import pandas as pd
from tqdm.notebook import tqdm

# Import custom modules
from src import process_jsons
from src import amzn
from src import chrome_driver


## 1.1 Setup the config parser.

In [2]:
config = configparser.ConfigParser()
config.read('../config.ini')


['../config.ini']

## 1.2 Import jsons and extract them to a list of order dictionaries.

- Each json file is comprised of one order. 

In [52]:
path_to_json = '../data/sample_json_orders' 
orders_dict_list = process_jsons.extract_jsons_from_directory(path_to_json)

In [60]:
# Filters the orders by a list of countries

country_filter_list = ['US']
filtered_orders = process_jsons.filter_json(orders_dict_list, country_filter_list)

# 3. Running the Script

In [61]:
driver = chrome_driver.setup()
amzn.login(driver, config['amazon_login']['email'], config['amazon_login']['password'])
#amzn.clear_cart(driver)

In [66]:
# Creates a dataframe to track how each order interacts with amazon. 
order_results_dict_list = list()

for i in tqdm(range(len(filtered_orders))):
    
    order_results_dict_list = amzn.add_items_to_cart(filtered_orders, driver, i, order_results_dict_list)
    order_results_df = pd.DataFrame(order_results_dict_list)
    

{'product_id': 'B07CMKX3C7', 'quantity': 2, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}
{'product_id': 'B07YTX2VLP', 'quantity': 1, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}
{'product_id': 'B07K39FRSL', 'quantity': 1, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}
One-time purchase:
{'product_id': 'B000XY9LT4', 'quantity': 2, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}
One-time purchase:
{'product_id': 'B0013L3XMM', 'quantity': 2, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}
Buy new:
{'product_id': 'B079QG5CDZ', 'quantity': 1, 'seller_selection_criteria': {'buy_box': True}, 'variants': []}



In [67]:
order_results_df = pd.DataFrame(order_results_dict_list)
order_results_df

,order_index,json_dict,product_id,product_quant,num_products,href_link,went_to_product_page,product_id_matches,discovery_method,num_results,select_quantity_exists,select_quantity_num_avail,add_to_cart_exsts,added_to_cart
0,0,"{'idempotency_key': '22ds4iQ423ZlvhR', 'shippi...",B07CMKX3C7,2,1,https://www.amazon.com/Tempur-Pedic-Adaptable-...,True,True,span,1,True,True,True,True
1,1,"{'idempotency_key': 'nttfkg3oiSJ7Yjd', 'shippi...",B07YTX2VLP,1,1,https://www.amazon.com/Christmas-Goodies-Draws...,True,True,span,1,False,None,True,False
2,2,"{'idempotency_key': 'Zr2zqnNJCM9UrT', 'shippin...",B07K39FRSL,1,1,https://www.amazon.com/Apple-Watch-GPS-38mm-Sp...,True,True,span,1,False,None,False,False
3,3,"{'idempotency_key': 'JhzFp36Qilg59I3', 'shippi...",B000XY9LT4,2,3,https://www.amazon.com/Eucerin-Intensive-Repai...,True,True,div,1,True,True,True,True
4,3,"{'idempotency_key': 'JhzFp36Qilg59I3', 'shippi...",B0013L3XMM,2,3,https://www.amazon.com/Eucerin-Sensitive-Redne...,True,True,span,1,True,True,True,True
5,3,"{'idempotency_key': 'JhzFp36Qilg59I3', 'shippi...",B079QG5CDZ,1,3,https://www.amazon.com/URBAN-ARMOR-GEAR-5-8-In...,True,True,span,1,True,True,True,True


In [65]:
amzn.clear_cart(driver)

Starting to clear cart ...
Checkout cart has been cleared of 8 items.


In [32]:
amzn.get_num_cart_items(driver)


0

In [68]:
pd.DataFrame(filtered_orders)

,idempotency_key,shipping_address,client_notes,payment_method,retailer,affiliate_info,products,shipping_method,billing_address,retailer_credentials,webhooks,max_price,client_token
0,22ds4iQ423ZlvhR,"{'first_name': 'Ohshop', 'last_name': 'Amazon ...","{'order_code': 'AFMAJO', 'retailer': 'amazon'}","{'name_on_card': 'Jason Kim', 'expiration_year...",amazon,{'tag': 'ohmyzip19-20'},"[{'product_id': 'B07CMKX3C7', 'quantity': 2, '...",cheapest,"{'first_name': 'Jason M', 'last_name': 'Kim', ...",{'email': 'join@ohmyzip.com'},{'order_placed': 'https://www.ohmyzip.com/zinc...,999900,08302263B276E420C6F32249
1,nttfkg3oiSJ7Yjd,"{'first_name': 'Ohshop', 'last_name': 'Amazon ...","{'order_code': 'ASAXDU', 'retailer': 'amazon'}","{'name_on_card': 'Jason Kim', 'expiration_year...",amazon,{'tag': 'ohmyzip19-20'},"[{'product_id': 'B07YTX2VLP', 'quantity': 1, '...",cheapest,"{'first_name': 'Jason M', 'last_name': 'Kim', ...",{'email': 'join@ohmyzip.com'},{'order_placed': 'https://www.ohmyzip.com/zinc...,999900,08302263B276E420C6F32249
2,Zr2zqnNJCM9UrT,"{'first_name': 'Ohshop', 'last_name': 'Amazon ...","{'order_code': 'AZ62GS', 'retailer': 'amazon'}","{'name_on_card': 'Jason Kim', 'expiration_year...",amazon,{'tag': 'ohmyzip19-20'},"[{'product_id': 'B07K39FRSL', 'quantity': 1, '...",cheapest,"{'first_name': 'Jason M', 'last_name': 'Kim', ...",{'email': 'join@ohmyzip.com'},{'order_placed': 'https://www.ohmyzip.com/zinc...,999900,08302263B276E420C6F32249
3,JhzFp36Qilg59I3,"{'first_name': 'Ohshop', 'last_name': 'Amazon ...","{'order_code': 'A9CR3X', 'retailer': 'amazon'}","{'name_on_card': 'Jason Kim', 'expiration_year...",amazon,{'tag': 'ohmyzip19-20'},"[{'product_id': 'B000XY9LT4', 'quantity': 2, '...",cheapest,"{'first_name': 'Jason M', 'last_name': 'Kim', ...",{'email': 'join@ohmyzip.com'},{'order_placed': 'https://www.ohmyzip.com/zinc...,999900,08302263B276E420C6F32249
